<a href="https://colab.research.google.com/github/yimuzy/Is20f/blob/master/xuan/Speed_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def BSM_characteristic_function(v, x0, T, r, sigma):
    cf_value = np.exp(((x0 / T + r - 0.5 * sigma ** 2) * 1j * v
                - 0.5 * sigma ** 2 * v ** 2) * T)
    return cf_value
def BSM_call_characteristic_function(v,alpha, x0, T, r, sigma):
    res=np.exp(-r*T)/((alpha+1j*v)*(alpha+1j*v+1))\
        *BSM_characteristic_function((v-(alpha+1)*1j), x0, T, r, sigma)
    return res
    
def SimpsonW(N,eta):
    delt = np.zeros(N, dtype=np.float)
    delt[0] = 1
    j = np.arange(1, N + 1, 1)
    SimpsonW = eta*(3 + (-1) ** j - delt) / 3
    return SimpsonW
    

def Simposon_numerical_integrate(S0, K, T, r, sigma):
    k = np.log(K)
    x0 = np.log(S0)
    N=1024
    B=153.6
    eta=B/N
    W=SimpsonW(N,eta)
    
    alpha=1.5
    sumx=0
    for j in range(N):
        v_j=j*eta
        temp=np.exp(-1j*v_j*k)*\
            BSM_call_characteristic_function(v_j,alpha, x0, T, r, sigma)*\
            W[j]            
        sumx+=temp.real

        
    return sumx*np.exp(-alpha*k)/np.pi

In [ ]:
S0 = 100.0  # index level
K = 108.52520983216910821762196480844  # option strike
T = 1.0  # maturity date
r = 0.0475  # risk-less short rate
sigma = 0.2  # volatility

print ('>>>>>>>>>>FT call value is ' + str(Simposon_numerical_integrate(S0, K, T, r, sigma)))

>>>>>>>>>>FT call value is 6.477779672276538


In [ ]:
%cd~

!git clone https://github.com/hhk54250/20MA573-HHK.git 
pass


/root
Cloning into '20MA573-HHK'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 200 (delta 11), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (200/200), 5.79 MiB | 9.00 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [ ]:

%cd 20MA573-HHK/src/
%ls

/root/20MA573-HHK/src
bsm.py  optiondata.dat  prj01.ipynb  prj02.ipynb


In [ ]:
from bsm import *


'''===============
Test bsm_price
================='''
gbm1 = Gbm(
    init_state = 100., 
    drift_ratio = .0475,
    vol_ratio = .2)
option1 = VanillaOption(
    otype = 1,
    strike = 108.52520983216910821762196480844,                
    maturity = 1.
)    

print('>>>>>>>>>>BSM call value is ' + str(gbm1.bsm_price(option1)))

>>>>>>>>>>BSM call value is 6.477779672277251


In [ ]:
def fft(FFTFunc):
    N=2**10
    eta=0.15
    lambda_ = 2 * np.pi / (N *eta)   
    t=np.arange(0, N, 1)
    sumy=np.asarray([np.sum(np.exp(-1j*lambda_*eta*t*m)*FFTFunc) for m in range(N)])

        
    return sumy

def BSM_call_value_FFT(S0, K, T, r, sigma):
    k = np.log(K)
    x0 = np.log(S0)
    N =2**10
    alpha=1.5
    
    eta=0.15
    lambda_ = 2 * np.pi / (N *eta)
    beta=x0-lambda_*N/2
    km=np.asarray([beta+i*lambda_ for i in range(N)])
    W=SimpsonW(N,eta)
    v=np.asarray([i*eta for i in range(N)])
    Psi=np.asarray([BSM_call_characteristic_function(vj,alpha, x0, T, r, sigma)  for vj in v])
    FFTFunc=Psi*np.exp(-1j*beta*v)*W
    
    
    y=fft(FFTFunc).real
    
    
    cT=np.exp(-alpha*km)*y/np.pi
    
    return cT

In [ ]:

S0 = 100.0  # index level
K = 110.0  # option strike
T = 1.0  # maturity date
r = 0.0475  # risk-less short rate
sigma = 0.2  # volatility
print('>>>>>>>>>>FFT call value is ' + str(BSM_call_value_FFT(S0, K, T, r, sigma)[514]))

>>>>>>>>>>FFT call value is 6.4777796722766245


In [ ]:
"FFT time test"
S0 = 100.0  # index level
K = 110.0  # option strike
T = 1.0  # maturity date
r = 0.0475  # risk-less short rate
sigma = 0.2  # volatility
%time BSM_call_value_FFT(S0, K, T, r, sigma)

CPU times: user 117 ms, sys: 1.99 ms, total: 119 ms
Wall time: 120 ms


array([-1.51437903e+14, -1.21413940e+14, -9.50754645e+13, ...,
       -1.33706636e-13, -1.13945211e-13, -9.53065405e-14])

In [ ]:
"FT time test"
S0 = 100.0  # index level
T = 1.0  # maturity date
r = 0.0475  # risk-less short rate
sigma = 0.2  # volatility
N =2**10 
eta=0.15
lambda_ = 2 * np.pi / (N *eta)
x0 = np.log(S0)
beta=x0-lambda_*N/2
k=np.asarray([np.e**(beta+lambda_*n) for n in range(N)])
%time np.asarray([Simposon_numerical_integrate(S0, k[n], T, r, sigma) for n in range(N)])


CPU times: user 13.2 s, sys: 8.89 ms, total: 13.3 s
Wall time: 13.3 s


array([-1.51437903e+14, -1.21413940e+14, -9.50754645e+13, ...,
       -1.33706636e-13, -1.13945211e-13, -9.53065405e-14])

In [ ]:
"BSM time test"
gbm1 = Gbm(
    init_state = 100., 
    drift_ratio = .0475,
    vol_ratio = .2)
option1 = VanillaOption(
    otype = 1,
    strike = k,                
    maturity = 1.
)    

%time gbm1.bsm_price(option1)

CPU times: user 1.78 ms, sys: 0 ns, total: 1.78 ms
Wall time: 1.54 ms


array([99.99999992, 99.99999992, 99.99999992, ...,  0.        ,
        0.        ,  0.        ])

In [ ]:
def BSM_call_value_NumpyFFT(S0, K, T, r, sigma):
    k = np.log(K)
    x0 = np.log(S0)
    N =2**10
    alpha=1.5
    
    eta=0.15
    lambda_ = 2 * np.pi / (N *eta)
    beta=x0-lambda_*N/2
    km=np.asarray([beta+i*lambda_ for i in range(N)])
    W=SimpsonW(N,eta)
    v=np.asarray([i*eta for i in range(N)])
    Psi=np.asarray([BSM_call_characteristic_function(vj,alpha, x0, T, r, sigma)  for vj in v])
    FFTFunc=Psi*np.exp(-1j*beta*v)*W
    
    
    y=np.fft.fft(FFTFunc).real
    
    
    cT=np.exp(-alpha*km)*y/np.pi

In [ ]:
"FFT time test using Numpy.FFT package"
S0 = 100.0  # index level
K = 110.0  # option strike
T = 1.0  # maturity date
r = 0.0475  # risk-less short rate
sigma = 0.2  # volatility
%time BSM_call_value_NumpyFFT(S0, K, T, r, sigma)

CPU times: user 15.3 ms, sys: 986 µs, total: 16.2 ms
Wall time: 16 ms
